In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel("GooglePlay_App_Data.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'GooglePlay_App_Data.xlsx'

# SECCIÓN A: Auditoría Inicial del Dataset

## A.1 Dimensiones del Dataset

In [ ]:
df.shape

El dataset contiene:
*   206 registros
*   12 variables

Es un dataset relativamente pequeño, lo cual implica:

*   No podemos permitir pérdida innecesaria de datos.
*   Las decisiones de eliminación deben estar bien justificadas.
*   Cada registro tiene peso en el análisis final.

## A.2 Estructura y Tipos de Datos

In [ ]:
df.info()

***review_title***

Tipo: float64,
Non-null: 0

*   Una columna de texto no puede ser float.
*   Además, está completamente vacía
*   Esto indica problema en el origen del archivo.

Esta columna es candidata fuerte a eliminación.

***thumbs_up***

Tipo: float64, Tiene valores nulos (161 de 206 completos)

Debería ser entero, no float.

***review_date y developer_response_date***

Tipo: object

Deberían ser datetime.

Esto puede causar errores en análisis temporal.

***developer_response***

Tiene valores nulos.

Puede ser normal (no siempre hay respuesta).

Pero debemos validar coherencia con developer_response_date.

***appVersion***

Tiene muchos valores nulos.

146 no nulos → 60 faltantes.

Posible problema de calidad o registros antiguos.

***rating***

Tipo correcto: int64

Pero aún debemos validar rango (debe ser 1–5).

***language_code (mal escrito como laguage_code)***

Error tipográfico en nombre de columna.

Esto debe corregirse.

## A.3 Valores Nulos

In [ ]:
df.isnull().sum()

***Problemas detectados:***

review_title → 206 nulos (100%)

thumbs_up → 45 nulos

review_date → 1 nulo

developer_response → 2 nulos

developer_response_date → 19 nulos

appVersion → 60 nulos

No es un dataset limpio.
Tiene inconsistencias estructurales claras.

## A.4 Duplicados

In [ ]:
df.duplicated().sum()

No existen registros completamente duplicados.

Esto es positivo.

Sin embargo, aún debemos verificar si hay review_id duplicados (clave primaria lógica).

## A.5 Validaciones Clave de Integridad

In [ ]:
df['rating'].unique()

Todas las calificaciones están dentro del rango esperado (1 a 5).

No existen valores inválidos como 0, 6 o negativos.

La variable está correctamente estructurada y no requiere depuración.

In [ ]:
df['review_id'].nunique()

Existen 206 IDs únicos.

Coincide exactamente con el total de registros.

No hay duplicados lógicos.

# SECCIÓN B: Limpieza Exhaustiva

## B.1 Eliminación de columna irrelevante

In [ ]:
df = df.drop(columns=['review_title'])

*   review_title tenía 100% valores nulos.
*   No aporta información analítica.
*   Mantenerla solo introduce ruido estructural.
*   Eliminarla mejora la calidad del dataset.

## B.2 Diagnóstico posterior a eliminación

In [ ]:
df.info()

206 registros

11 columnas

Persisten problemas de tipado y nulos

## B.3 Corrección del nombre de columna mal escrito

In [ ]:
df = df.rename(columns={'laguage_code': 'language_code'})

In [ ]:
df.columns

Existía un error ortográfico en el nombre de la columna.

## B.4 Corrección de Tipos de Datos

In [ ]:
df.dtypes

*   review_date → object
*   developer_response_date → object
*   thumbs_up (Float → Entero)




Problema detectado:

Las columnas de fecha estaban en formato object debido a:

*   Mezcla de tipos internos (str + datetime + float)
*   Caracteres invisibles en algunos registros

### Diagnóstico de review_date

In [ ]:
df['review_date'].apply(type).value_counts()

187 valores tipo str

18 valores tipo datetime

1 valor tipo float (NaN real)

Conclusión:
Excel almacenó fechas con distintos tipos internos.

In [ ]:
df['review_date'] = df['review_date'].apply(
    lambda x: x.replace('\xa0', '').strip() if isinstance(x, str) else x
)

Se detectó que algunos strings no convertían correctamente a datetime debido a caracteres invisibles (ej. \xa0) y espacios adicionales.

Se aplicó limpieza únicamente a los valores tipo string.

In [ ]:
mask = df['review_date'].apply(lambda x: isinstance(x, str))

df.loc[mask, 'review_date'] = pd.to_datetime(
    df.loc[mask, 'review_date'],
    format='%m/%d/%Y %H:%M',
    errors='coerce'
)

Todos los strings válidos fueron convertidos correctamente.

In [ ]:
df['review_date'] = pd.to_datetime(
    df['review_date'],
    errors='coerce'
)

In [ ]:
df.dtypes

Aunque los valores ya estaban convertidos, la columna seguía como object por mezcla interna.

Se aplicó conversión global final.

In [ ]:
df['review_date'].isna().sum()

1 valor nulo real (confirmado previamente en Excel)


### Diagnóstico de developer_response_date

In [ ]:
df['developer_response_date'].apply(type).value_counts()

Se identificó mezcla similar de tipos.

In [ ]:
df['developer_response_date'] = df['developer_response_date'].apply(
    lambda x: x.replace('\xa0', '').strip() if isinstance(x, str) else x
)

Limpieza

In [ ]:
mask_dev = df['developer_response_date'].apply(lambda x: isinstance(x, str))

df.loc[mask_dev, 'developer_response_date'] = pd.to_datetime(
    df.loc[mask_dev, 'developer_response_date'],
    format='%m/%d/%Y %H:%M',
    errors='coerce'
)

Conversión selectiva

In [ ]:
df['developer_response_date'] = pd.to_datetime(
    df['developer_response_date'],
    errors='coerce'
)

Conversión final unificada

In [ ]:
df.dtypes

Validación

In [ ]:
df['developer_response_date'].isna().sum()

Múltiples nulos reales (confirmado previamente en Excel)


### Corrección de thumbs_up (Float → Entero)

In [ ]:
df.info()

Tipo: float64

45 valores nulos

Debería ser entero

Representa conteo (no puede ser decimal)

In [ ]:
df['thumbs_up'] = df['thumbs_up'].fillna(0)
df['thumbs_up'] = df['thumbs_up'].astype(int)

df['thumbs_up'].info()

La variable thumbs_up se encontraba tipada como float debido a la presencia de valores nulos.
Dado que representa un conteo de interacciones, se decidió imputar los valores faltantes con 0 y convertir la variable a tipo entero.

Esto garantiza consistencia numérica y evita distorsiones en análisis posteriore

## B.5 Tratamiento Integral de Valores Nulos (Eliminación e Imputación)

### Identificación de valores nulos

In [ ]:
df.isnull().sum()

review_date presenta 1 valor nulo.

developer_response_date presenta 20 valores nulos.

appVersion presenta 60 valores nulos.

developer_response presenta 2 valores nulos.

### Eliminación de registros críticos

In [ ]:
df = df[df['review_date'].notna()]
df.shape

Se eliminó el único registro que no contenía fecha de reseña (review_date), ya que esta variable es crítica para el análisis temporal y métricas derivadas.
El impacto sobre el volumen total de datos es mínimo y mejora la consistencia estructural del dataset.

### Imputación Estratégica de Valores Nulos

In [ ]:
df['developer_response'] = df['developer_response'].fillna("No Response")
df['appVersion'] = df['appVersion'].fillna("Unknown")

Los valores faltantes en developer_response representan ausencia de respuesta y fueron imputados como "No Response" para mantener coherencia semántica.
Los valores faltantes en appVersion fueron reemplazados por "Unknown" para evitar pérdida de información y facilitar segmentaciones en dashboards.

### Creación de variable estructural de respuesta

In [ ]:
df['has_response'] = (df['developer_response'] != "No Response").astype(int)

1 → el desarrollador respondió

0 → no respondió

Se creó la variable binaria has_response con el objetivo de modelar explícitamente la gestión del desarrollador frente a las reseñas.

Esta variable permite:

*   Calcular tasa de respuesta.
*   Analizar si se responde más a reseñas negativas.
*   Medir impacto de la respuesta en el engagement (thumbs_up).
*   Evitar depender de valores nulos en análisis posteriores.


### Validación final de nulos

In [ ]:
df.isnull().sum()

review_date → 0

developer_response → 0

appVersion → 0

developer_response_date → mantiene NaT solo donde no hubo respuesta

## B.6 Validación de Coherencia Temporal

### Detección de incoherencias

In [ ]:
df[
    (df['has_response'] == 1) &
    (df['developer_response_date'] < df['review_date'])
]

In [ ]:
df.loc[
    df['developer_response_date'] < df['review_date'],
    'developer_response_date'
] = pd.NaT

Se detectaron registros donde la fecha de respuesta del desarrollador era anterior a la fecha de reseña.

Dado que el análisis no contempla métricas de tiempo de respuesta, se decidió invalidar dichas fechas estableciéndolas como NaT para mantener coherencia temporal sin eliminar registros.

# SECCIÓN C: Estadísticas Descriptivas

In [ ]:
df.describe()

## C.1 Interpretación de df.describe()

***Rating***
*   Media: 3.98
*   Mediana: 5
*   Mínimo: 1
*   Máximo: 5
*   Desviación estándar: 1.35

La distribución del rating muestra una tendencia positiva, con una media cercana a 4 y una mediana de 5 estrellas, lo que sugiere una percepción mayoritariamente favorable de la aplicación.


***Thumbs Up***
*   Media: 1.52
*   Mediana: 0
*   Mínimo: 1
*   Máximo: 31
*   Desviación: 3.36

La variable thumbs_up presenta una distribución altamente sesgada, donde la mayoría de reseñas no reciben votos, aunque existen casos aislados con alta interacción.


***Review Date***

Desde 2016 hasta 2024.
*   El dataset cubre un período amplio (8 años).
*   Permite análisis temporal si se desea.


***Developer Response Date***
*   Count: 183 de 205
*   Eso significa que hay 22 sin fecha válida (lo que ya se entiende).


***Has Response***

Media: 0.99

Se observa una tasa de respuesta del desarrollador superior al 98%, lo que sugiere una política activa de atención al usuario.

## C.2 Graficos descriptivos

### Distribución de Rating

In [ ]:
rating_counts = df['rating'].value_counts().sort_index()

plt.figure()
plt.bar(rating_counts.index, rating_counts.values)
plt.xlabel("Rating")
plt.ylabel("Cantidad de Reseñas")
plt.title("Distribución de Rating")
plt.show()

La distribución de ratings muestra una clara concentración en la calificación máxima (5 estrellas), con más de la mitad de las reseñas ubicadas en este nivel. Las calificaciones intermedias (3 y 4) y negativas (1 y 2) presentan una frecuencia considerablemente menor.

Esto sugiere una percepción general altamente positiva de la aplicación, con baja proporción de insatisfacción explícita por parte de los usuarios.

### Distribución de Thumbs Up

In [ ]:
plt.figure()
plt.hist(df['thumbs_up'], bins=20)
plt.xlabel("Thumbs Up")
plt.ylabel("Frecuencia")
plt.title("Distribución de Thumbs Up")
plt.show()

La variable thumbs_up presenta una distribución altamente concentrada en el valor cero, lo que indica que la mayoría de las reseñas no recibe interacción adicional por parte de otros usuarios.

Sin embargo, existen algunos casos aislados con mayor número de votos, lo que sugiere que ciertas reseñas generan mayor visibilidad o relevancia dentro de la comunidad.

### Reviews por Año

In [ ]:
df['year'] = df['review_date'].dt.year
reviews_per_year = df['year'].value_counts().sort_index()

plt.figure()
plt.plot(reviews_per_year.index, reviews_per_year.values)
plt.xlabel("Año")
plt.ylabel("Cantidad de Reviews")
plt.title("Reviews por Año")
plt.show()

El volumen de reseñas alcanza su punto máximo en 2016, seguido de una disminución sostenida hasta 2019. Posteriormente, se observa un comportamiento variable con ligeros repuntes en 2020 y 2022, aunque sin recuperar los niveles iniciales.

Esto podría indicar un período inicial de alta adopción o lanzamiento activo de la aplicación, seguido de una estabilización en la generación de reseñas en los años posteriores.

In [ ]:
df.to_csv("dataset_limpio_final.csv", index=False)